In [1]:
%matplotlib inline
import pandas as pd
import re
import numpy as np

# Get The Data Item Only

In [2]:
data_path='../../data/'
data_raw_path="../../data/raw/for third propose/"
data_man_path="../../data/manipulate/for third propose/"
model_path="../../model/modeling_third_propose/"

In [3]:
dtesting=pd.read_csv(data_man_path+"test_gocommerce_items_only_cln.csv")
dtag=pd.read_csv(data_man_path+"data_tag_goshop3_unique.csv").drop_duplicates()

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# dtesting

In [5]:
dtesting2=dtesting[~(dtesting.item_list_cln1.isnull())]
dtesting2=dtesting2[~(dtesting2.item_list_cln2.isnull())]

# Label with real (rule based)

In [6]:
dtag["num_word"]=dtag.Keywords.apply(lambda x : len(x.split(" ")))
dtag["num_word_tag"]=dtag.Tags.apply(lambda x : len(x.split(" ")))

keyword_set=set(dtag.Keywords.unique())
keyword_set1=set([t for t in keyword_set if len(t.split(" "))==1])
keyword_set2=set([t for t in keyword_set if len(t.split(" "))==2])
keyword_set3=set([t for t in keyword_set if len(t.split(" "))==3])


ts=set(dtag.Tags.unique())
ts1=set([t for t in ts if len(t.split(" "))==1])
ts2=set([t for t in ts if len(t.split(" "))==2])
ts3=set([t for t in ts if len(t.split(" "))==3])

dict_keyword=pd.DataFrame(dtag.Tags)
dict_keyword.index=dtag.Keywords
dict_keyword=dict_keyword["Tags"].to_dict()

In [7]:
def include_tags(X,Y):
    X2=[]
    for x1 in X:
        k=0
        for y1 in Y:
            if x1 in y1:
                k=1
                break
        if k==0:
            X2.append(x1)
    
    return set(X2)|Y

def remove_term(text,X):
    for term in X:
        text=text.replace(' '+term+' '," ")
        text=re.sub( '\s+', ' ',text)
    return text
    

def tags(tx):
    X3=set([tg for tg in keyword_set3 if ' '+tg+' ' in ' '+tx+' '])
    tx2=remove_term(' '+tx+' ',X3)
    X3=set([dict_keyword[t] for t in X3])
    
    X2=set([tg for tg in keyword_set2 if ' '+tg+' ' in ' '+tx2+' '])
    tx3=remove_term(' '+tx2+' ',X2)
    X2=set([dict_keyword[t] for t in X2])

    X1=keyword_set1&set(tx3.split(" "))
    X1=set([dict_keyword[t] for t in X1])
    
    X_tags0=X1|X2|X3
    
    T1=set([t for t in X_tags0 if len(t.split(" "))==1])
    T2=set([t for t in X_tags0 if len(t.split(" "))==2])
    T3=set([t for t in X_tags0 if len(t.split(" "))==3])
    
    T12=include_tags(T1,T2)
    X_tags=include_tags(T12,T3)
    
    return X_tags

def tags_mult(X):
    XX=X.split(",")
    tg_X=set([])
    for t in XX:
        if len(t)>=3:
            t2=t.strip()
            tg_X=tg_X|tags(t2)
            
    return tuple(tg_X)

In [8]:
X=dtesting2.item_list_cln2

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.metrics import hamming_loss
from sklearn.preprocessing import LabelEncoder
from skmultilearn.problem_transform import LabelPowerset,ClassifierChain
from skmultilearn.adapt import MLkNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from gensim.models import Word2Vec
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors
from collections import defaultdict
from sklearn.preprocessing import RobustScaler
from sklearn.externals import joblib

# DL2 Model 

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import model_from_json

Using TensorFlow backend.


In [11]:
multilabel_binarizer_DL2=joblib.load(model_path+"multilabel_bin_for_DL2.pkl")
tokenizer_DL2=joblib.load(model_path+"tokenizer_for_DL2.pkl")

In [12]:
## Transform
# sequence encode
encoded_docs_DL2 = tokenizer_DL2.texts_to_sequences(X)
# pad sequences
max_length = 80
X2_DL2 = pad_sequences(encoded_docs_DL2, maxlen=max_length, padding='post')

In [13]:
# load json and create model
json_file = open(model_path+"wv_DL2.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
DL2_model = model_from_json(loaded_model_json)
# load weights into new model
DL2_model.load_weights(model_path+"wv_DL2.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
DL2_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Loaded model from disk


In [14]:
y_pred_DL2 = DL2_model.predict(X2_DL2) #Benchmark 0.16

In [15]:
def transform_y(yp,a=0.22):
    YY=(yp>a).astype(int)
    while YY.sum()==0 and a>=0.35:
        a=a-0.02
        YY=(yp>a).astype(int)
    return YY

In [16]:
y_pred2_DL2=np.array([transform_y(yp,a=0.55) for yp in y_pred_DL2])
dtesting2["pred_tag_DL2"]=multilabel_binarizer_DL2.inverse_transform(y_pred2_DL2)

In [17]:
gt_dl2=open(model_path+"good_tags_DL2.txt",'r')
gt_DL2=gt_dl2.readline().split(", ")

In [18]:
len(gt_DL2)

619

In [19]:
dtesting2.to_csv(data_man_path+"model_result_on_item_only_testing.csv",index=False,sep=';')
# dtesting2=pd.read_csv(data_man_path+"model_result_on_data_testing.csv")

In [20]:
dtesting2

,booking_time,order_no,merchant_name,type,item_id,price,item_list,item_list_cln1,item_list_cln2,pred_tag_DL2
0,2018-06-22 04:13:41.033434,F-38838821,A&W,food,31158681,7500.0,perkedel,perkedel,perkedel,"(perkedel,)"
1,2018-07-28 06:41:19.540410,F-61962406,A&W,food,31154567,7000.0,perkedel,perkedel,perkedel,"(perkedel,)"
2,2018-09-23 09:45:36.268406,F-99347083,A&W,food,31157276,7500.0,perkedel,perkedel,perkedel,"(perkedel,)"
3,2018-09-28 04:40:25.801412,F-102281166,A&W,food,31165095,7000.0,perkedel,perkedel,perkedel,"(perkedel,)"
4,2018-08-01 04:48:32.141758,F-64344114,A&W,food,31163524,7000.0,perkedel,perkedel,perkedel,"(perkedel,)"
5,2018-07-17 08:00:04.541248,F-55530580,A&W,food,31157416,7500.0,perkedel,perkedel,perkedel,"(perkedel,)"
6,2018-09-09 15:23:29.254722,F-90394328,A&W,food,31161198,7500.0,perkedel,perkedel,perkedel,"(perkedel,)"
7,2018-07-21 08:51:14.111594,F-57868244,A&W,food,31162646,7500.0,perkedel,perkedel,perkedel,"(perkedel,)"
8,2018-09-17 05:42:55.508280,F-95193076,A&W,food,30846275,7000.0,perkedel,perkedel,perkedel,"(perkedel,)"
9,2018-09-21 06:22:53.952331,F-97762949,A&W,food,31169825,7500.0,perkedel,perkedel,perkedel,"(perkedel,)"


# Evaluation

In [21]:
dtesting2[["item_list","real_tags","pred_tag_DL1","pred_tag_DL2"]].sample(15)

KeyError: "['real_tags' 'pred_tag_DL1'] not in index"

In [ ]:
((dd_eval_DL2.AUC>0.8)&(dd_eval_DL2["learn_rate(%)"]>0)).mean()*451

In [ ]:
dd_eval_DL2[(dd_eval_DL2.AUC>0.8)&(dd_eval_DL2["learn_rate(%)"]>0)][["accuracy","AUC"]].describe()

In [ ]:
dtesting2.head()

In [ ]:
dtesting2_upload=dtesting2[["order_no","merchant_name","type","item_list","real_tags","pred_tag_DL1","pred_tag_DL2"]]

In [ ]:
dtesting2_upload["real_tags"]=dtesting2_upload["real_tags"].apply(lambda x:', '.join(x))
dtesting2_upload["pred_tag_DL1"]=dtesting2_upload["pred_tag_DL1"].apply(lambda x:', '.join(x))
dtesting2_upload["pred_tag_DL2"]=dtesting2_upload["pred_tag_DL2"].apply(lambda x:', '.join(x))